In [3]:
from itertools import combinations

import os
import sys
from pathlib import Path

sys.path.append(str((Path(os.path.abspath("")) / "..").resolve()))
from sudoku_variants import Sudoku, SudokuAI, SudokuConst
from sudoku_variants.rule import Orthogonal, SubBoard, Knight, King, Consecutive, Jigsaw
from sudoku_variants.func import rules as R
from sudoku_variants.helper.profile import profile


def standard_rules():
    return [Orthogonal(), SubBoard()]


def variant_rules():
    return [Knight(), King(), Consecutive(), Jigsaw()]

### AI

In [4]:
@profile(10)
def run(rules):
    board = [[0 for _ in range(SudokuConst.NUM_COL)] for _ in range(SudokuConst.NUM_ROW)]
    sudoku = Sudoku(board, rules)
    SudokuAI.solve(sudoku)


for length in range(len(variant_rules()) + 1):
    for subset in combinations(variant_rules(), length):
        print(R.to_name(list(subset)))
        rules = R.with_standard_rules(list(subset))

        # board = [[0 for _ in range(SudokuConst.NUM_COL)] for _ in range(SudokuConst.NUM_ROW)]
        # sudoku = Sudoku(board, rules)
        # R.populate_initial_data(rules)
        # SudokuAI.solve(sudoku)
        # R.extract_data_from_board(rules, sudoku.board)

        run(rules)


Time: 0.0308s, 10x | 0.0031s/time
Knight
Time: 1.9358s, 10x | 0.1936s/time
King
Time: 0.2360s, 10x | 0.0236s/time
Consecutive
Time: 0.0296s, 10x | 0.0030s/time
Jigsaw
Time: 0.0299s, 10x | 0.0030s/time
Knight, King
Time: 1.6608s, 10x | 0.1661s/time
Knight, Consecutive
Time: 1.8338s, 10x | 0.1834s/time
Knight, Jigsaw
Time: 1.5579s, 10x | 0.1558s/time
King, Consecutive
Time: 0.0484s, 10x | 0.0048s/time
King, Jigsaw
Time: 0.0744s, 10x | 0.0074s/time
Consecutive, Jigsaw
Time: 0.0290s, 10x | 0.0029s/time
Knight, King, Consecutive
Time: 2.0964s, 10x | 0.2096s/time
Knight, King, Jigsaw
Time: 1.9349s, 10x | 0.1935s/time
Knight, Consecutive, Jigsaw
Time: 1.1424s, 10x | 0.1142s/time
King, Consecutive, Jigsaw
Time: 0.0830s, 10x | 0.0083s/time
Knight, King, Consecutive, Jigsaw
Time: 2.2907s, 10x | 0.2291s/time


In [3]:
@profile(10)
def run(rules, to_erase):
    SudokuAI.generate(rules, max_erased=to_erase)


for to_erase in [30, 40, 50, 60]:
    print(to_erase)
    rules = [Orthogonal(), SubBoard()]
    run(rules, to_erase)

35
Time: 0.2784s, 10x | 0.0278s/time
40
Time: 0.4221s, 10x | 0.0422s/time
50
Time: 2.7139s, 10x | 0.2714s/time
60
Time: 252.0264s, 10x | 25.2026s/time


In [4]:
@profile(1)
def run():
    sudoku = SudokuAI.generate([Orthogonal(), SubBoard(), Knight(), King(), Consecutive()], max_erased=40)
    print(sudoku)


run()

Applied rules: Orthogonal, SubBoard, Knight, King, Consecutive
+-------+-------+-------+
| * 9 * | 5 3 * | 6 * 2 |
| 6 * 2 | 4 * * | 5 3 1 |
| 5 * * | 6 * * | 4 9 * |
+-------+-------+-------+
| * * * | 7 * * | 1 * 8 |
| 1 6 * | 2 * * | * 5 * |
| 7 5 3 | * * 8 | 2 * 9 |
+-------+-------+-------+
| 8 2 * | 9 * 5 | * 1 * |
| * * * | 8 * * | 9 7 5 |
| 9 7 * | 3 1 * | * * * |
+-------+-------+-------+
Time: 0.2183s, 1x | 0.2183s/time
